In [110]:
PROJECT = "emart-datafabric"
DATASET = "common_dev"
TABLE = "dfm_sample_eapp_data"
DT = "2022-08-30"
LIMIT = 30

REDIS_INFO = {
    "host": "redis-master.redis-farm.svc.cluster.local",
    "port": "6379",
}

In [37]:
from pydatafabric.gcp import get_spark, bq_to_df
spark = get_spark(extra_jars="gs://emart-datafabric-resources/jars/spark-redis_2.12-2.4.2.jar")

df = bq_to_df(f"""
    select  *
    from  `{PROJECT}.{DATASET}.{TABLE}`
    WHERE dt = '{DT}'
    limit {LIMIT}
""", spark_session=spark)

22/10/05 16:00:55 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/10/05 16:00:55 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/10/05 16:00:55 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/10/05 16:00:55 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [38]:
spark.conf.set("spark.redis.host", REDIS_INFO["host"])
spark.conf.set("spark.redis.port", REDIS_INFO["port"])
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', 'true')

In [39]:
spark.conf.get("spark.redis.host")

'redis-master.redis-farm.svc.cluster.local'

In [40]:
df.show(10)

+----------------+--------------------+---------+---------+---+----------------+--------+-------------+-----+---------------------------------+---------------+-------------+--------------------------------+----------+----------+-----------+-----------+----------+----------+-------------+-------------+-------------+------------------+-------------+------------------+-------+-------+-------------+------------+--------------+-----------+-----------+---------+----------+-----------+--------------------+-------------+
|       review_id|             cust_id|gender_cd|gender_nm|age|agrde_cd_10_unit|store_cd|     store_nm|score|                         comments|comments_length|       sku_cd|                          sku_nm|  order_dt|        dt|prdt_cat_cd|prdt_cat_nm|prdt_di_cd|prdt_di_nm|prdt_gcode_cd|prdt_gcode_nm|prdt_mcode_cd|     prdt_mcode_nm|prdt_dcode_cd|     prdt_dcode_nm|area_cd|area_nm|    longitude|    latitude|comments_point|image_point|thumb_point|action_cd|blind_flag|active_fla

In [41]:
df.createOrReplaceTempView(f"temp_{TABLE}")

In [42]:
result_df = spark.sql(
    f'select * from temp_{TABLE} limit 1')

result_df.show(10)

+----------------+--------------------+---------+---------+---+----------------+--------+-------------+-----+--------------------------+---------------+-------------+----------------------------+----------+----------+-----------+-----------+----------+----------+-------------+-------------+-------------+-------------+-------------+------------------+-------+-------+-------------+------------+--------------+-----------+-----------+---------+----------+-----------+--------------------+-------------+
|       review_id|             cust_id|gender_cd|gender_nm|age|agrde_cd_10_unit|store_cd|     store_nm|score|                  comments|comments_length|       sku_cd|                      sku_nm|  order_dt|        dt|prdt_cat_cd|prdt_cat_nm|prdt_di_cd|prdt_di_nm|prdt_gcode_cd|prdt_gcode_nm|prdt_mcode_cd|prdt_mcode_nm|prdt_dcode_cd|     prdt_dcode_nm|area_cd|area_nm|    longitude|    latitude|comments_point|image_point|thumb_point|action_cd|blind_flag|active_flag|            tag_list|avg_tag_s

In [43]:
result_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- cust_id: string (nullable = true)
 |-- gender_cd: string (nullable = true)
 |-- gender_nm: string (nullable = true)
 |-- age: string (nullable = true)
 |-- agrde_cd_10_unit: string (nullable = true)
 |-- store_cd: string (nullable = true)
 |-- store_nm: string (nullable = true)
 |-- score: long (nullable = true)
 |-- comments: string (nullable = true)
 |-- comments_length: long (nullable = true)
 |-- sku_cd: string (nullable = true)
 |-- sku_nm: string (nullable = true)
 |-- order_dt: date (nullable = true)
 |-- dt: date (nullable = true)
 |-- prdt_cat_cd: string (nullable = true)
 |-- prdt_cat_nm: string (nullable = true)
 |-- prdt_di_cd: string (nullable = true)
 |-- prdt_di_nm: string (nullable = true)
 |-- prdt_gcode_cd: string (nullable = true)
 |-- prdt_gcode_nm: string (nullable = true)
 |-- prdt_mcode_cd: string (nullable = true)
 |-- prdt_mcode_nm: string (nullable = true)
 |-- prdt_dcode_cd: string (nullable = true)
 |-- prdt

In [44]:
if result_df.count() > 0:
    result_df.write.format("org.apache.spark.sql.redis") \
    .option("table", "test_dfm_sample_eapp_data") \
    .option("key.column", "review_id") \
    .save()

Py4JJavaError: An error occurred while calling o732.save.
: java.lang.NoClassDefFoundError: redis/clients/jedis/PipelineBase
	at org.apache.spark.sql.redis.DefaultSource.createRelation(DefaultSource.scala:21)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:46)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:90)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.spark.sql.execution.Spark

In [45]:
spark.stop()

In [26]:
import redis
import json

# 레디스 연결
rd = redis.StrictRedis(host=REDIS_INFO["host"], port=REDIS_INFO["port"], db=0)

In [31]:
# dict 데이터 선언
data = {
    "key1": "테스트값1",
    "key2": "테스트값2",
    "key3": "테스트값3",
    "key4": "테스트값4",
}

In [32]:

json_data_dict = json.dumps(data, ensure_ascii=False).encode('utf-8')

# 데이터 set
rd.set("dict", json_data_dict)

True

In [33]:
# 데이터 get
result = rd.get("dict")
result = result.decode('utf-8')

# json loads 
result = dict(json.loads(result))

In [34]:
print(result)

{'key1': '테스트값1', 'key2': '테스트값2', 'key3': '테스트값3', 'key4': '테스트값4'}


In [56]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

In [57]:
sparkConf = SparkConf()
sparkConf.set("spark.jars", "gs://emart-datafabric-resources/jars/spark-redis_2.12-2.4.2.jar")

In [83]:
sparkConf.set("spark.redis.host", REDIS_INFO['host'])
sparkConf.set("spark.redis.port", REDIS_INFO['port'])
sparkConf.set("spark.redis.auth", "")
sparkConf.set('spark.sql.execution.arrow.pyspark.enabled', 'true')

In [99]:
sparkConf.getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.app.name', 'emart-x223726-959e04fb-642e-4bae-abb8-16ffb524b719'),
 ('spark.repl.local.jars',
  'file:/tmp/spark-496a5e59-37d3-4fdc-85be-3b89f99b096a/spark-bigquery-with-dependencies_2.12-latest.jar,file:/tmp/spark-496a5e59-37d3-4fdc-85be-3b89f99b096a/spark-redis_2.12-3.1.0.jar'),
 ('spark.redis.port', '6379'),
 ('spark.yarn.am.memory', '640m'),
 ('spark.port.maxRetries', '128'),
 ('spark.history.fs.logDirectory',
  'gs://dataproc-temp-asia-northeast3-149277781422-nozf8jsl/afff9a2a-167f-4877-a5f6-2dd534eea3ab/spark-job-history'),
 ('spark.redis.host', 'redis-master.redis-farm.svc.cluster.local'),
 ('spark.driver.memory', '6g'),
 ('spark.executor.instances', '2'),
 ('spark.dynamicAllocation.maxExecutors', '200'),
 ('spark.yarn.unmanagedAM.enabled', 'true'),
 ('spark.driver.host', '198.19.39.8'),
 ('spark.executor.memory', '8g'),
 ('spark.yarn.historyServer.address', 'yellowelephant-m:18080'),
 (

In [102]:
spark = SparkSession.builder.config(conf=sparkConf).enableHiveSupport().getOrCreate()

In [104]:
spark.sql(f""" select 1 as num """).registerTempTable('test_table')

In [105]:
df = spark.sql("""
    select
        *
    from 
        test_table
""")

In [107]:
df.show(10)

+---+
|num|
+---+
|  1|
+---+



In [108]:
df.write.format("org.apache.spark.sql.redis").option("table", "test_table_test").option("key.column", "num").save(mode="overwrite")

Py4JJavaError: An error occurred while calling o2542.save.
: java.lang.NoClassDefFoundError: redis/clients/jedis/PipelineBase
	at org.apache.spark.sql.redis.DefaultSource.createRelation(DefaultSource.scala:21)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:46)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:90)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.spark.sql.execution.Spar

In [109]:
spark.stop()